In [4]:
import pandas as pd
import sqlite3
import re

In [ ]:
'''
dados base:
('07601-345', 'Avenida Hum', "Olho D'Água", 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)

dados depois da traformação:
(7601, 'Avenida Hum', "Olho D'Água", 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, '2024-07-24')

realizar a quebra na primeira coluna, talvez um cast quebrando esse dado em duas novas colunas do cep 
ou remover o traço no meio que é contabilizado como string e não integer
 
'''

In [45]:
bd = 'banco_cep.db'
banco_caminho = f'/workspaces/projeto_ing/INGESTION/database/{bd}'
conn = sqlite3.connect(banco_caminho)
cursor = conn.cursor()
query = 'select * from tabela_cepavenidamairipora'
cursor.execute(query)
resultado = cursor.fetchall()
for linha in resultado:
    print(linha)


('07601-345', 'Avenida Hum', "Olho D'Água", 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07613-690', 'Avenida Rubi', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07613-655', 'Avenida Ouro', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07627-160', 'Avenida Berna', 'Luiz Fagundes', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07657-210', 'Avenida Alpes', 'Mato Dentro', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07613-680', 'Avenida Safira', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07631-320', 'Avenida Sandra', 'Rio acima', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07634-020', 'Avenida Brasil', 'Lavras', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07617-005', 'Avenida Acesso', 'Juqueri Mirim', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07631-570', 'Avenida do Sol', 'Rio acima', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671)
('07663-080', 'Avenida Brasil', 'Jundiaizinho (Terra Preta)', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671

In [48]:
df = pd.DataFrame(resultado, columns=['cep', 'logradouro', 'bairro', 'localidade', 'uf', 'ibge', 'gia', 'ddd', 'siafi'])
# print(df.head())
df_ccep = df['cep'].str.replace(r'-', '')
df_cep =df['cep'].str.extract(r'(\d{5})-\d{3}')
df_cep_compl = df['cep'].str.extract(r'(-\d{3})')
df['ccep'] = df_ccep
df['ccep_loc'] = df_cep
df['cep_compl'] = df_cep_compl
df_rplc = df['cep_compl'].str.replace('-', '')
df['ccep_compl'] = df_rplc
columns = ['cep', 'cep_compl']
df.drop(columns, inplace=True, axis=1)
df['ccep'].astype(int)
df['ccep_loc'].astype(int)
df['ccep_compl'].astype(int)
print(df.head())

      logradouro         bairro localidade  uf     ibge   gia  ddd  siafi  \
0    Avenida Hum    Olho D'Água  Mairiporã  SP  3528502  4339   11   6671   
1   Avenida Rubi    Mantiqueira  Mairiporã  SP  3528502  4339   11   6671   
2   Avenida Ouro    Mantiqueira  Mairiporã  SP  3528502  4339   11   6671   
3  Avenida Berna  Luiz Fagundes  Mairiporã  SP  3528502  4339   11   6671   
4  Avenida Alpes    Mato Dentro  Mairiporã  SP  3528502  4339   11   6671   

       ccep ccep_loc ccep_compl  
0  07601345    07601        345  
1  07613690    07613        690  
2  07613655    07613        655  
3  07627160    07627        160  
4  07657210    07657        210  


In [49]:
df.to_sql(name='ttbela_slver_cep', con=conn, if_exists = 'append', index = False)
print('tabela ingerida')


tabela ingerida


In [ ]:
#ajustar o ultimo campo, date está quebrando

In [50]:
query = 'select * from ttbela_slver_cep'
cursor.execute(query)
resultado = cursor.fetchall()
for linha in resultado:
    print(linha)


('Avenida Hum', "Olho D'Água", 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7601345, 7601, 345, None)
('Avenida Rubi', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7613690, 7613, 690, None)
('Avenida Ouro', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7613655, 7613, 655, None)
('Avenida Berna', 'Luiz Fagundes', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7627160, 7627, 160, None)
('Avenida Alpes', 'Mato Dentro', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7657210, 7657, 210, None)
('Avenida Safira', 'Mantiqueira', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7613680, 7613, 680, None)
('Avenida Sandra', 'Rio acima', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7631320, 7631, 320, None)
('Avenida Brasil', 'Lavras', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7634020, 7634, 20, None)
('Avenida Acesso', 'Juqueri Mirim', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 7617005, 7617, 5, None)
('Avenida do Sol', 'Rio acima', 'Mairiporã', 'SP', 3528502, 4339, 11, 6671, 763

In [28]:
file = '/workspaces/projeto_ing/INGESTION/app/src/backp/resultado_cepavenidamairipora_20240711_121430.txt'
df = pd.read_csv(file, delimiter=';', usecols=lambda column: column !='Unnamed: 0')
df_ccep = df['cep'].str.replace(r'-', '')
df_cep =df['cep'].str.extract(r'(\d{5})-\d{3}')
df_cep_compl = df['cep'].str.extract(r'(-\d{3})')
df['ccep'] = df_ccep
df['cep_loc'] = df_cep
df['cep_compl'] = df_cep_compl
df_rplc = df['cep_compl'].str.replace('-', '')
df['ccep_compl'] = df_rplc
columns = ['cep', 'cep_compl']
df.drop(columns, inplace=True, axis=1)
df['ccep'].astype(int)
df['cep_loc'].astype(int)
df['ccep_compl'].astype(int)
display(df)
# df.describe()
df.dtypes


,logradouro,bairro,localidade,uf,ibge,gia,ddd,siafi,ccep,cep_loc,ccep_compl
0,Avenida Hum,Olho D'Água,Mairiporã,SP,3528502,4339,11,6671,07601345,07601,345
1,Avenida Rubi,Mantiqueira,Mairiporã,SP,3528502,4339,11,6671,07613690,07613,690
2,Avenida Ouro,Mantiqueira,Mairiporã,SP,3528502,4339,11,6671,07613655,07613,655
3,Avenida Berna,Luiz Fagundes,Mairiporã,SP,3528502,4339,11,6671,07627160,07627,160
4,Avenida Alpes,Mato Dentro,Mairiporã,SP,3528502,4339,11,6671,07657210,07657,210
5,Avenida Safira,Mantiqueira,Mairiporã,SP,3528502,4339,11,6671,07613680,07613,680
6,Avenida Sandra,Rio acima,Mairiporã,SP,3528502,4339,11,6671,07631320,07631,320
7,Avenida Brasil,Lavras,Mairiporã,SP,3528502,4339,11,6671,07634020,07634,020
8,Avenida Acesso,Juqueri Mirim,Mairiporã,SP,3528502,4339,11,6671,07617005,07617,005
9,Avenida do Sol,Rio acima,Mairiporã,SP,3528502,4339,11,6671,07631570,07631,570


logradouro    object
bairro        object
localidade    object
uf            object
ibge           int64
gia            int64
ddd            int64
siafi          int64
ccep          object
cep_loc       object
ccep_compl    object
dtype: object

In [ ]:
            # src_arquivo = os.path.join(path, nome_arquivo)       
            # df = pd.read_csv(f'{src_arquivo}', delimiter=';', usecols=lambda column: column !='Unnamed: 0')
            # file_bckp = f'{src_arquivo}'
            # df_ccep = df['cep'].str.replace(r'-', '')
            # df_cep =df['cep'].str.extract(r'(\d{5})-\d{3}')
            # df_cep_compl = df['cep'].str.extract(r'(-\d{3})')
            # df['ccep'] = df_ccep
            # df['cep_loc'] = df_cep
            # df['cep_compl'] = df_cep_compl
            # df_rplc = df['cep_compl'].str.replace('-', '')
            # df['ccep_compl'] = df_rplc
            # columns = ['cep', 'cep_compl']
            # df.drop(columns, inplace=True, axis=1)
            # df['ccep'].astype(int)
            # df['cep_loc'].astype(int)
            # df['ccep_compl'].astype(int)